In [305]:
import os

# windows記得先載visual studio c++
import cv2
import json
# import openslide
import matplotlib.pyplot as plt
import large_image
import numpy as np
import skimage.io
from skimage import io
# import large_image_source_openslide
from PIL import Image
# from openslide.deepzoom import DeepZoomGenerator
import matplotlib.image as mpimg
from tqdm import tqdm_notebook
import math
import pandas as pd
import MTM  # multi template matching
from MTM import matchTemplates, drawBoxesOnRGB
from shapely.geometry import Polygon
from scipy import stats
import glob
from pandas import json_normalize
from tqdm import tqdm

Image.MAX_IMAGE_PIXELS = 10000000000

In [143]:
df = pd.read_excel("./template_excel/ellipse_detail_BFC_0712_new.xlsx")

In [144]:
df = df.drop("Unnamed: 0",axis=1)

In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71041 entries, 0 to 71040
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mass_name        71041 non-null  object 
 1   img_name         71041 non-null  object 
 2   threshold        71041 non-null  int64  
 3   center           71041 non-null  object 
 4   axle_short       71041 non-null  float64
 5   axle_long        71041 non-null  float64
 6   ratio            71041 non-null  float64
 7   angle            71041 non-null  float64
 8   area_cv2         71041 non-null  float64
 9   perimeter        71041 non-null  float64
 10  contours_pixel   71041 non-null  float64
 11  contours_number  71041 non-null  int64  
 12  type             71041 non-null  object 
 13  calcularity      71041 non-null  float64
dtypes: float64(8), int64(2), object(4)
memory usage: 7.6+ MB


In [146]:
df1 = pd.read_excel("./template_excel/convex_detail_PTC_20230712_new.xlsx")

In [147]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61703 entries, 0 to 61702
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mass_name        61703 non-null  object 
 1   img_name         61703 non-null  object 
 2   threshold        61703 non-null  int64  
 3   center           61703 non-null  object 
 4   axle_short       61703 non-null  float64
 5   axle_long        61703 non-null  float64
 6   ratio            61703 non-null  float64
 7   angle            61703 non-null  float64
 8   area_cv2         61703 non-null  float64
 9   perimeter        61703 non-null  float64
 10  contours_pixel   61703 non-null  float64
 11  contours_number  61703 non-null  int64  
 12  type             61703 non-null  object 
 13  calcularity      61703 non-null  float64
dtypes: float64(8), int64(2), object(4)
memory usage: 6.6+ MB


In [148]:
merge = pd.concat([df,df1])
merge = merge.reset_index(drop=True)

In [149]:
merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132744 entries, 0 to 132743
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   mass_name        132744 non-null  object 
 1   img_name         132744 non-null  object 
 2   threshold        132744 non-null  int64  
 3   center           132744 non-null  object 
 4   axle_short       132744 non-null  float64
 5   axle_long        132744 non-null  float64
 6   ratio            132744 non-null  float64
 7   angle            132744 non-null  float64
 8   area_cv2         132744 non-null  float64
 9   perimeter        132744 non-null  float64
 10  contours_pixel   132744 non-null  float64
 11  contours_number  132744 non-null  int64  
 12  type             132744 non-null  object 
 13  calcularity      132744 non-null  float64
dtypes: float64(8), int64(2), object(4)
memory usage: 14.2+ MB


In [309]:
label = pd.DataFrame()
with open('./Thyroid_AI_data_20220103.json', 'r') as f:
    label = json_normalize(json.load(f))

In [314]:
label_df = pd.DataFrame()
for i in range(len(label)):
    for j in range(len(label['targets'][i])):
        try:
            data = {
                "slide_name" : f'{label["slide_name"][i]}',
                "mass_name" : f'{label["slide_name"][i]}_{j}',
                "label" : f'{label["targets"][i][j]["labels"][0].split(":")[1]}'
            }
        except:
            data = {
                "slide_name" : f'{label["slide_name"][i]}',
                "mass_name" : f'{label["slide_name"][i]}_{j}',
                "label" : f'{label["targets"][i][j]["labels"]}'
            }
        label_df = pd.concat([label_df,pd.DataFrame([data])],ignore_index=True)

In [315]:
label_df['label'].value_counts()

label
PTC              23041
FCs              11409
Histiocytes        403
HurthleCell        219
Indeterminate        9
[]                   2
Name: count, dtype: int64

In [152]:
merge = pd.merge(merge,label_df,on="mass_name",how="left")

In [154]:
merge['label'].value_counts()

label
FCs              70994
PTC              61665
HurthleCell         52
Indeterminate       33
Name: count, dtype: int64

In [153]:
merge['type'].value_counts()

type
PTC       61703
Normal    29948
HT        28751
CYST      12342
Name: count, dtype: int64

In [321]:
train_data = pd.concat([merge[merge['type']=="PTC"].sample(48000),merge[merge['type']=="Normal"].sample(24000),merge[merge['type']=="HT"].sample(22400),merge[merge['type']=="CYST"].sample(9600)])
# train_data = pd.concat([merge[merge['label']=="PTC"].sample(48000),merge[merge['label']=="FCs"].sample(56000),merge[merge['label']=="HurthleCell"].sample(40)])

In [322]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 104000 entries, 120882 to 10325
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   mass_name        104000 non-null  object 
 1   img_name         104000 non-null  object 
 2   threshold        104000 non-null  int64  
 3   center           104000 non-null  object 
 4   axle_short       104000 non-null  float64
 5   axle_long        104000 non-null  float64
 6   ratio            104000 non-null  float64
 7   angle            104000 non-null  float64
 8   area_cv2         104000 non-null  float64
 9   perimeter        104000 non-null  float64
 10  contours_pixel   104000 non-null  float64
 11  contours_number  104000 non-null  int64  
 12  type             104000 non-null  object 
 13  calcularity      104000 non-null  float64
 14  slide_name       104000 non-null  object 
 15  label            104000 non-null  object 
dtypes: float64(8), int64(2), object(6)
memo

In [323]:
no_train = merge.drop(train_data.index).reset_index(drop=True)

In [324]:
test_data = pd.concat([no_train[no_train['type']=="PTC"].sample(6000),no_train[no_train['type']=="Normal"].sample(3000),no_train[no_train['type']=="HT"].sample(2800),no_train[no_train['type']=="CYST"].sample(1200)])
# test_data = pd.concat([no_train[no_train['label']=="PTC"],no_train[no_train['label']=="FCs"],no_train[no_train['label']=="HurthleCell"].sample(6)])

In [325]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13000 entries, 22773 to 2764
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mass_name        13000 non-null  object 
 1   img_name         13000 non-null  object 
 2   threshold        13000 non-null  int64  
 3   center           13000 non-null  object 
 4   axle_short       13000 non-null  float64
 5   axle_long        13000 non-null  float64
 6   ratio            13000 non-null  float64
 7   angle            13000 non-null  float64
 8   area_cv2         13000 non-null  float64
 9   perimeter        13000 non-null  float64
 10  contours_pixel   13000 non-null  float64
 11  contours_number  13000 non-null  int64  
 12  type             13000 non-null  object 
 13  calcularity      13000 non-null  float64
 14  slide_name       13000 non-null  object 
 15  label            13000 non-null  object 
dtypes: float64(8), int64(2), object(6)
memory usage: 1.7+ MB


In [326]:
predict_data = no_train.drop(test_data.index)

In [327]:
predict_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15744 entries, 6 to 28743
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mass_name        15744 non-null  object 
 1   img_name         15744 non-null  object 
 2   threshold        15744 non-null  int64  
 3   center           15744 non-null  object 
 4   axle_short       15744 non-null  float64
 5   axle_long        15744 non-null  float64
 6   ratio            15744 non-null  float64
 7   angle            15744 non-null  float64
 8   area_cv2         15744 non-null  float64
 9   perimeter        15744 non-null  float64
 10  contours_pixel   15744 non-null  float64
 11  contours_number  15744 non-null  int64  
 12  type             15744 non-null  object 
 13  calcularity      15744 non-null  float64
 14  slide_name       15744 non-null  object 
 15  label            15744 non-null  object 
dtypes: float64(8), int64(2), object(6)
memory usage: 2.0+ MB


In [328]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from PIL import Image

# Define the custom CellImageClassifier class
class CellImageClassifier:
    def __init__(self, image_width, image_height, num_channels, num_classes):
        self.image_width = image_width
        self.image_height = image_height
        self.num_channels = num_channels
        self.num_classes = num_classes
        # self.model_5 = self.build_model_5()
        self.model_4 = self.build_model_4()
        self.model_3_128 = self.build_model_3_128()
        self.model_3_64 = self.build_model_3_64()
        self.model_3_32 = self.build_model_3_32()
        

    def build_model_4(self):
        model = models.Sequential()
        model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(self.image_width, self.image_height, self.num_channels)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(16, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),
                                                                                  tf.keras.metrics.Recall(),tf.keras.metrics.TruePositives(thresholds=0),
                                                                                  tf.keras.metrics.TrueNegatives(thresholds=0),
                                                                                  tf.keras.metrics.FalsePositives(thresholds=0),
                                                                                  tf.keras.metrics.FalseNegatives(thresholds=0),
                                                                                  tf.keras.metrics.PrecisionAtRecall(recall=0.5),
                                                                                  tf.keras.metrics.SensitivityAtSpecificity(specificity=0.5),
                                                                                  tf.keras.metrics.SpecificityAtSensitivity(sensitivity=0.5)])
        return model
    
    def build_model_3_128(self):
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.image_width, self.image_height, self.num_channels)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(128, (3, 3), activation='relu'))  # Additional Convolutional layer
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))  # Additional Fully connected layer
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),
                                                                                  tf.keras.metrics.Recall(),tf.keras.metrics.TruePositives(thresholds=0),
                                                                                  tf.keras.metrics.TrueNegatives(thresholds=0),
                                                                                  tf.keras.metrics.FalsePositives(thresholds=0),
                                                                                  tf.keras.metrics.FalseNegatives(thresholds=0),
                                                                                  tf.keras.metrics.PrecisionAtRecall(recall=0.5),
                                                                                  tf.keras.metrics.SensitivityAtSpecificity(specificity=0.5),
                                                                                  tf.keras.metrics.SpecificityAtSensitivity(sensitivity=0.5)])
        return model
    
    def build_model_3_64(self):
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(self.image_width, self.image_height, self.num_channels)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),
                                                                                  tf.keras.metrics.Recall(),tf.keras.metrics.TruePositives(thresholds=0),
                                                                                  tf.keras.metrics.TrueNegatives(thresholds=0),
                                                                                  tf.keras.metrics.FalsePositives(thresholds=0),
                                                                                  tf.keras.metrics.FalseNegatives(thresholds=0),
                                                                                  tf.keras.metrics.PrecisionAtRecall(recall=0.5),
                                                                                  tf.keras.metrics.SensitivityAtSpecificity(specificity=0.5),
                                                                                  tf.keras.metrics.SpecificityAtSensitivity(sensitivity=0.5)])
        return model
    
    def build_model_3_32(self):
        model = models.Sequential()
        model.add(layers.Conv2D(8, (3, 3), activation='relu', input_shape=(self.image_width, self.image_height, self.num_channels)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(16, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(layers.Flatten())
        model.add(layers.Dense(32, activation='relu'))
        model.add(layers.Dense(self.num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),
                                                                                  tf.keras.metrics.Recall(),tf.keras.metrics.TruePositives(thresholds=0),
                                                                                  tf.keras.metrics.TrueNegatives(thresholds=0),
                                                                                  tf.keras.metrics.FalsePositives(thresholds=0),
                                                                                  tf.keras.metrics.FalseNegatives(thresholds=0),
                                                                                  tf.keras.metrics.PrecisionAtRecall(recall=0.5),
                                                                                  tf.keras.metrics.SensitivityAtSpecificity(specificity=0.5),
                                                                                  tf.keras.metrics.SpecificityAtSensitivity(sensitivity=0.5)])
        return model

    def preprocess_image(self, image_path):
        img = Image.open(image_path)
        img = img.resize((self.image_width, self.image_height))
        img_array = np.array(img)
        img_array = img_array / 255.0  # Normalize pixel values between 0 and 1
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
        return img_array

    def train_4(self, train_images, train_labels, num_epochs, batch_size):
        self.model_4.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size)
    def train_3_128(self, train_images, train_labels, num_epochs, batch_size):
        self.model_3_128.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size)
    def train_3_64(self, train_images, train_labels, num_epochs, batch_size):
        self.model_3_64.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size)
    def train_3_32(self, train_images, train_labels, num_epochs, batch_size):
        self.model_3_32.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size)
    
    def evaluate_4(self, test_images, test_labels):
        test_results = self.model_4.evaluate(test_images, test_labels)
        print('Test Loss:', test_results[0])
        print('Test Accuracy:', test_results[1])
        print('Test AUC:', test_results[2])
        print('Test Presicion:', test_results[3])
        print('Test Recall:', test_results[4])
        print('Test TP:', test_results[5])
        print('Test TN:', test_results[6])
        print('Test FP:', test_results[7])
        print('Test FN:', test_results[8])
        print('Test PrecisionAtRecall:', test_results[9])
        print('Test SensitivityAtSpecificity:', test_results[10])
        print('Test SpecificityAtSensitivity:', test_results[11])
        
    def evaluate_3_128(self, test_images, test_labels):
        test_results = self.model_3_128.evaluate(test_images, test_labels)
        print('Test Loss:', test_results[0])
        print('Test Accuracy:', test_results[1])
        print('Test AUC:', test_results[2])
        print('Test Presicion:', test_results[3])
        print('Test Recall:', test_results[4])
        print('Test TP:', test_results[5])
        print('Test TN:', test_results[6])
        print('Test FP:', test_results[7])
        print('Test FN:', test_results[8])
        print('Test PrecisionAtRecall:', test_results[9])
        print('Test SensitivityAtSpecificity:', test_results[10])
        print('Test SpecificityAtSensitivity:', test_results[11])
        
    def evaluate_3_64(self, test_images, test_labels):
        test_results = self.model_3_64.evaluate(test_images, test_labels)
        print('Test Loss:', test_results[0])
        print('Test Accuracy:', test_results[1])
        print('Test AUC:', test_results[2])
        print('Test Presicion:', test_results[3])
        print('Test Recall:', test_results[4])
        print('Test TP:', test_results[5])
        print('Test TN:', test_results[6])
        print('Test FP:', test_results[7])
        print('Test FN:', test_results[8])
        print('Test PrecisionAtRecall:', test_results[9])
        print('Test SensitivityAtSpecificity:', test_results[10])
        print('Test SpecificityAtSensitivity:', test_results[11])
        
    def evaluate_3_32(self, test_images, test_labels):
        test_results = self.model_3_32.evaluate(test_images, test_labels)
        print('Test Loss:', test_results[0])
        print('Test Accuracy:', test_results[1])
        print('Test AUC:', test_results[2])
        print('Test Presicion:', test_results[3])
        print('Test Recall:', test_results[4])
        print('Test TP:', test_results[5])
        print('Test TN:', test_results[6])
        print('Test FP:', test_results[7])
        print('Test FN:', test_results[8])
        print('Test PrecisionAtRecall:', test_results[9])
        print('Test SensitivityAtSpecificity:', test_results[10])
        print('Test SpecificityAtSensitivity:', test_results[11])
    
    def predict_4(self, image_path):
        preprocessed_image = self.preprocess_image(image_path)
        prediction = self.model_4.predict(preprocessed_image)
        predicted_class_index = np.argmax(prediction)
        return predicted_class_index
    
    def predict_3_128(self, image_path):
        preprocessed_image = self.preprocess_image(image_path)
        prediction = self.model_3_128.predict(preprocessed_image)
        predicted_class_index = np.argmax(prediction)
        return predicted_class_index
    
    def predict_3_64(self, image_path):
        preprocessed_image = self.preprocess_image(image_path)
        prediction = self.model_3_64.predict(preprocessed_image)
        predicted_class_index = np.argmax(prediction)
        return predicted_class_index
    
    def predict_3_32(self, image_path):
        preprocessed_image = self.preprocess_image(image_path)
        prediction = self.model_3_32.predict(preprocessed_image)
        predicted_class_index = np.argmax(prediction)
        return predicted_class_index

In [329]:
# Define dataset paths and parameters
image_width = 64
image_height = 64
num_channels = 3
num_classes = 5
num_epochs = 10
batch_size = 32

In [330]:
# Load and preprocess the dataset
train_images = []
train_labels = []
test_images = []
test_labels = []

for imagename in train_data['img_name']:
    imagename = imagename+".png"
    train_data_dir = "./all_image/all_singal/"
    image = Image.open(os.path.join(train_data_dir, imagename))
    image = image.resize((image_width, image_height))
    image_array = np.array(image) / 255.0
    train_images.append(image_array)
    if "CYST" in imagename:
        train_labels.append(0)
    elif "HT" in imagename:
        train_labels.append(1)
    elif "PTC" in imagename:
        train_labels.append(2)
    elif "HCN" in imagename:
        train_labels.append(3)
    else:
        train_labels.append(4)

In [331]:
for imagename in test_data['img_name']:
    imagename = imagename+".png"
    test_data_dir = "./all_image/all_singal/"
    image = Image.open(os.path.join(test_data_dir, imagename))
    image = image.resize((image_width, image_height))
    image_array = np.array(image) / 255.0
    test_images.append(image_array)
    if "CYST" in imagename:
        test_labels.append(0)
    elif "HT" in imagename:
        test_labels.append(1)
    elif "PTC" in imagename:
        test_labels.append(2)
    elif "HCN" in imagename:
        test_labels.append(3)
    else:
        test_labels.append(4)

# 5 layer

In [222]:
train_images = np.array(train_images)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_images = np.array(test_images)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Create and train the cell image classifier
classifier = CellImageClassifier(image_width, image_height, num_channels, num_classes)
classifier.train_4(train_images, train_labels, num_epochs, batch_size)

# Evaluate the classifier
classifier.evaluate_4(test_images, test_labels)

Epoch 1/10
3252/3252 [==============================] - 411s 126ms/step - loss: 0.5984 - accuracy: 0.7167 - auc_32: 0.9486 - precision_32: 0.8578 - recall_32: 0.5773 - true_positives_32: 104040.0000 - true_negatives_32: 0.0000e+00 - false_positives_32: 416160.0000 - false_negatives_32: 0.0000e+00 - precision_at_recall_32: 0.9375 - sensitivity_at_specificity_32: 0.9991 - specificity_at_sensitivity_32: 0.9917
Epoch 2/10
3252/3252 [==============================] - 501s 154ms/step - loss: 0.5094 - accuracy: 0.7671 - auc_32: 0.9632 - precision_32: 0.8394 - recall_32: 0.6833 - true_positives_32: 104040.0000 - true_negatives_32: 0.0000e+00 - false_positives_32: 416160.0000 - false_negatives_32: 0.0000e+00 - precision_at_recall_32: 0.9725 - sensitivity_at_specificity_32: 0.9993 - specificity_at_sensitivity_32: 0.9964
Epoch 3/10
3252/3252 [==============================] - 514s 158ms/step - loss: 0.4679 - accuracy: 0.7907 - auc_32: 0.9694 - precision_32: 0.8357 - recall_32: 0.7366 - true_posit

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [223]:
# Predict on new images
df_predict = pd.DataFrame()
for i in predict_data['img_name']:
    new_image_path = f'./all_image/all_singal/{i}.png'
    predicted_class_index = classifier.predict_4(new_image_path)

    # Map class index to class label (assuming binary classification)
    class_labels = ['CYST', 'HT', 'PTC', 'HCN', 'Normal']
    predicted_class_label = class_labels[predicted_class_index]
    predict = {"image_name":i,"predict_type":predicted_class_label}
    
    df_predict = pd.concat([df_predict,pd.DataFrame([predict])],ignore_index=True)

In [224]:
df_predict

,image_name,predict_type
0,BFC_20_002_98_2,Normal
1,BFC_20_002_98_3,Normal
2,BFC_20_002_98_5,Normal
3,BFC_20_002_98_6,Normal
4,BFC_20_002_98_11,Normal
5,BFC_20_002_98_16,CYST
6,BFC_20_002_98_19,Normal
7,BFC_20_003_10_7,HT
8,BFC_20_003_10_8,Normal
9,BFC_20_003_10_12,Normal


In [ ]:
df_predict.loc[df_predict["image_name"].str.contains("CYST"),"real_type"] = "CYST"
df_predict.loc[df_predict["image_name"].str.contains("HT"),"real_type"] = "HT"
df_predict.loc[df_predict["image_name"].str.contains("PTC"),"real_type"] = "PTC"
df_predict.loc[df_predict["real_type"].isna(),"real_type"] = "Normal"

In [ ]:
df_predict.loc[df_predict["predict_type"]==df_predict["real_type"],"correct"] = "Y"
df_predict.loc[df_predict["correct"].isna(),"correct"] = "N"

In [ ]:
df_predict['correct'].value_counts()

correct
Y    21
N    12
Name: count, dtype: int64

In [ ]:
df_predict['mass']=df_predict['image_name'].apply(lambda x : "_".join(x.split("_")[0:-1]))

In [ ]:
df_predict['slide']=df_predict['mass'].apply(lambda x : "_".join(x.split("_")[0:-1]))

In [ ]:
group = df_predict.groupby(['mass','predict_type']).size().reset_index(name='count')

In [ ]:
group

,mass,predict_type,count
0,BFC_20_002_98,CYST,1
1,BFC_20_002_98,Normal,6
2,BFC_20_003_10,HT,8
3,BFC_20_003_10,Normal,4
4,BFC_20_003_60,HT,3
5,BFC_20_003_60,Normal,6
6,BFC_20_023HT_9,HT,5


In [ ]:
category_max = group.groupby('mass').apply(lambda x: x.loc[x['count'].idxmax()])

In [ ]:
category_max = category_max[['mass', 'predict_type', 'count']].reset_index(drop=True)

In [ ]:
category_max.loc[category_max["mass"].str.contains("CYST"),"real_type"] = "CYST"
category_max.loc[category_max["mass"].str.contains("HT"),"real_type"] = "HT"
category_max.loc[category_max["mass"].str.contains("PTC"),"real_type"] = "PTC"
category_max.loc[category_max["real_type"].isna(),"real_type"] = "Normal"

In [ ]:
category_max.loc[category_max["predict_type"]==category_max["real_type"],"correct"] = "Y"
category_max.loc[category_max["correct"].isna(),"correct"] = "N"

In [ ]:
category_max

,mass,predict_type,count,real_type,correct
0,BFC_20_002_98,Normal,6,Normal,Y
1,BFC_20_003_10,HT,8,Normal,N
2,BFC_20_003_60,Normal,6,Normal,Y
3,BFC_20_023HT_9,HT,5,HT,Y


In [ ]:
category_max['correct'].value_counts()

correct
Y    3
N    1
Name: count, dtype: int64

In [ ]:
category_max['slide']=category_max['mass'].apply(lambda x : "_".join(x.split("_")[0:-1]))

In [ ]:
category_max

,mass,predict_type,count,real_type,correct,slide
0,BFC_20_002_98,Normal,6,Normal,Y,BFC_20_002
1,BFC_20_003_10,HT,8,Normal,N,BFC_20_003
2,BFC_20_003_60,Normal,6,Normal,Y,BFC_20_003
3,BFC_20_023HT_9,HT,5,HT,Y,BFC_20_023HT


In [ ]:
final_result = category_max.groupby(['slide','predict_type']).size().reset_index(name='count')

In [ ]:
final_result_max = final_result.groupby('slide').apply(lambda x: x.loc[x['count'].idxmax()])

In [ ]:
final_result_max.loc[final_result_max["slide"].str.contains("CYST"),"real_type"] = "CYST"
final_result_max.loc[final_result_max["slide"].str.contains("HT"),"real_type"] = "HT"
final_result_max.loc[final_result_max["slide"].str.contains("PTC"),"real_type"] = "PTC"
final_result_max.loc[final_result_max["real_type"].isna(),"real_type"] = "Normal"

In [ ]:
final_result_max

,slide,predict_type,count,real_type
slide,,,,
BFC_20_002,BFC_20_002,Normal,1,Normal
BFC_20_003,BFC_20_003,HT,1,Normal
BFC_20_023HT,BFC_20_023HT,HT,1,HT


In [ ]:
final_result_max.loc[final_result_max["predict_type"]==final_result_max["real_type"],"correct"] = "Y"
final_result_max.loc[final_result_max["correct"].isna(),"correct"] = "N"

In [ ]:
final_result_max['correct'].value_counts()

correct
Y    2
N    1
Name: count, dtype: int64

In [246]:
2/3

0.6666666666666666

In [65]:
150/156

0.9615384615384616

In [66]:
final_result_max[final_result_max['correct']=="N"]

,slide,predict_type,count,real_type,correct
slide,,,,,
BFC_20_006HU,BFC_20_006HU,HT,11,Normal,N
BFC_20_027,BFC_20_027,HT,13,Normal,N
BFC_20_029FN,BFC_20_029FN,HT,5,Normal,N
BFC_20_035CYST,BFC_20_035CYST,Normal,21,CYST,N
BFC_20_059_C,BFC_20_059_C,HT,29,Normal,N
BFC_20_068_HCN_C,BFC_20_068_HCN_C,HT,4,Normal,N


In [68]:
df_predict.to_excel("./df_predict.xlsx",index=False)

# 3 layer 128

In [81]:
train_images = np.array(train_images)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_images = np.array(test_images)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Create and train the cell image classifier
classifier = CellImageClassifier(image_width, image_height, num_channels, num_classes)
classifier.train_3_128(train_images, train_labels, num_epochs, batch_size)

# Evaluate the classifier
classifier.evaluate_3_128(test_images, test_labels)

Epoch 1/10
2038/3250 [=================>............] - ETA: 3:58 - loss: 0.5926 - accuracy: 0.7192 - auc_2: 0.9327 - precision_2: 0.8558 - recall_2: 0.5771 - true_positives_2: 65216.0000 - true_negatives_2: 0.0000e+00 - false_positives_2: 195648.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9343 - sensitivity_at_specificity_2: 0.9991 - specificity_at_sensitivity_2: 0.9882

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



3103/3250 [===========================>..] - ETA: 28s - loss: 0.4780 - accuracy: 0.7840 - auc_2: 0.9571 - precision_2: 0.8356 - recall_2: 0.7205 - true_positives_2: 99296.0000 - true_negatives_2: 368.0000 - false_positives_2: 297520.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9795 - sensitivity_at_specificity_2: 0.9998 - specificity_at_sensitivity_2: 0.9965

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



3250/3250 [==============================] - 589s 181ms/step - loss: 0.4342 - accuracy: 0.8072 - auc_2: 0.9649 - precision_2: 0.8418 - recall_2: 0.7652 - true_positives_2: 104000.0000 - true_negatives_2: 3272.0000 - false_positives_2: 308728.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9889 - sensitivity_at_specificity_2: 0.9998 - specificity_at_sensitivity_2: 0.9981
Epoch 4/10
 596/3250 [====>.........................] - ETA: 9:16 - loss: 0.4125 - accuracy: 0.8177 - auc_2: 0.9683 - precision_2: 0.8458 - recall_2: 0.7805 - true_positives_2: 19072.0000 - true_negatives_2: 1065.0000 - false_positives_2: 56151.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9922 - sensitivity_at_specificity_2: 0.9997 - specificity_at_sensitivity_2: 0.9987

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



3250/3250 [==============================] - 677s 208ms/step - loss: 0.4046 - accuracy: 0.8230 - auc_2: 0.9698 - precision_2: 0.8496 - recall_2: 0.7910 - true_positives_2: 104000.0000 - true_negatives_2: 7384.0000 - false_positives_2: 304616.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9914 - sensitivity_at_specificity_2: 0.9997 - specificity_at_sensitivity_2: 0.9985
Epoch 5/10
1181/3250 [=========>....................] - ETA: 7:16 - loss: 0.3768 - accuracy: 0.8369 - auc_2: 0.9738 - precision_2: 0.8593 - recall_2: 0.8106 - true_positives_2: 37792.0000 - true_negatives_2: 5478.0000 - false_positives_2: 107898.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9946 - sensitivity_at_specificity_2: 0.9997 - specificity_at_sensitivity_2: 0.9991

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



3250/3250 [==============================] - 672s 207ms/step - loss: 0.3774 - accuracy: 0.8368 - auc_2: 0.9737 - precision_2: 0.8588 - recall_2: 0.8104 - true_positives_2: 104000.0000 - true_negatives_2: 18911.0000 - false_positives_2: 293089.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9945 - sensitivity_at_specificity_2: 0.9998 - specificity_at_sensitivity_2: 0.9991
Epoch 6/10
3250/3250 [==============================] - 587s 180ms/step - loss: 0.3636 - accuracy: 0.8446 - auc_2: 0.9757 - precision_2: 0.8629 - recall_2: 0.8224 - true_positives_2: 104000.0000 - true_negatives_2: 25507.0000 - false_positives_2: 286493.0000 - false_negatives_2: 0.0000e+00 - precision_at_recall_2: 0.9955 - sensitivity_at_specificity_2: 0.9997 - specificity_at_sensitivity_2: 0.9992
Epoch 7/10
3250/3250 [==============================] - 617s 190ms/step - loss: 0.3511 - accuracy: 0.8514 - auc_2: 0.9773 - precision_2: 0.8686 - recall_2: 0.8314 - true_positives_2: 104000.0000 - true_negativ

In [88]:
# Predict on new images
df_predict_3_128 = pd.DataFrame()
for i in predict_data['img_name']:
    new_image_path = f'./all_image/all_singal/{i}.png'
    predicted_class_index = classifier.predict_3_128(new_image_path)

    # Map class index to class label (assuming binary classification)
    class_labels = ['CYST', 'HT', 'PTC', 'Normal']
    predicted_class_label = class_labels[predicted_class_index]
    predict = {"image_name":i,"predict_type":predicted_class_label}
    
    df_predict_3_128 = pd.concat([df_predict_3_128,pd.DataFrame([predict])],ignore_index=True)

In [90]:
df_predict_3_128['predict_type'].value_counts()

predict_type
Normal    15724
PTC          13
HT            7
Name: count, dtype: int64

In [105]:
df_predict_3_128.to_excel("./df_predict_3_128.xlsx",index=False)

# 3 layer 64

In [332]:
train_images = np.array(train_images)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_images = np.array(test_images)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Create and train the cell image classifier
classifier = CellImageClassifier(image_width, image_height, num_channels, num_classes)
classifier.train_3_64(train_images, train_labels, num_epochs, batch_size)

# Evaluate the classifier
classifier.evaluate_3_64(test_images, test_labels)

Epoch 1/10
3250/3250 [==============================] - 680s 209ms/step - loss: 0.5801 - accuracy: 0.7320 - auc_50: 0.9525 - precision_50: 0.8450 - recall_50: 0.6106 - true_positives_50: 104000.0000 - true_negatives_50: 0.0000e+00 - false_positives_50: 416000.0000 - false_negatives_50: 0.0000e+00 - precision_at_recall_50: 0.9462 - sensitivity_at_specificity_50: 0.9990 - specificity_at_sensitivity_50: 0.9929
Epoch 2/10
 752/3250 [=====>........................] - ETA: 8:42 - loss: 0.5107 - accuracy: 0.7671 - auc_50: 0.9632 - precision_50: 0.8416 - recall_50: 0.6843 - true_positives_50: 24064.0000 - true_negatives_50: 0.0000e+00 - false_positives_50: 96256.0000 - false_negatives_50: 0.0000e+00 - precision_at_recall_50: 0.9740 - sensitivity_at_specificity_50: 0.9992 - specificity_at_sensitivity_50: 0.9967

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



1994/3250 [=================>............] - ETA: 4:16 - loss: 0.3955 - accuracy: 0.8281 - auc_50: 0.9784 - precision_50: 0.8548 - recall_50: 0.7961 - true_positives_50: 63808.0000 - true_negatives_50: 9562.0000 - false_positives_50: 245670.0000 - false_negatives_50: 0.0000e+00 - precision_at_recall_50: 0.9938 - sensitivity_at_specificity_50: 0.9997 - specificity_at_sensitivity_50: 0.9992

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



3250/3250 [==============================] - 667s 205ms/step - loss: 0.3945 - accuracy: 0.8283 - auc_50: 0.9785 - precision_50: 0.8548 - recall_50: 0.7966 - true_positives_50: 104000.0000 - true_negatives_50: 15354.0000 - false_positives_50: 400646.0000 - false_negatives_50: 0.0000e+00 - precision_at_recall_50: 0.9936 - sensitivity_at_specificity_50: 0.9997 - specificity_at_sensitivity_50: 0.9992
Epoch 6/10
 463/3250 [===>..........................] - ETA: 9:20 - loss: 0.3729 - accuracy: 0.8384 - auc_50: 0.9808 - precision_50: 0.8627 - recall_50: 0.8119 - true_positives_50: 14816.0000 - true_negatives_50: 2952.0000 - false_positives_50: 56312.0000 - false_negatives_50: 0.0000e+00 - precision_at_recall_50: 0.9960 - sensitivity_at_specificity_50: 0.9996 - specificity_at_sensitivity_50: 0.9995

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [333]:
# Predict on new images
df_predict_3_64 = pd.DataFrame()
for i in predict_data['img_name']:
    new_image_path = f'./all_image/all_singal/{i}.png'
    predicted_class_index = classifier.predict_3_64(new_image_path)

    # Map class index to class label (assuming binary classification)
    class_labels = ['CYST', 'HT', 'PTC', 'HCN', 'Normal']
    predicted_class_label = class_labels[predicted_class_index]
    predict = {"image_name":i,"predict_type":predicted_class_label}
    
    df_predict_3_64 = pd.concat([df_predict_3_64,pd.DataFrame([predict])],ignore_index=True)

In [294]:
merge

,mass_name,img_name,threshold,center,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,type,calcularity,slide_name,label
0,BFC_20_015HT_44,BFC_20_015HT_44_1,130,"(138.39991760253906, 162.02569580078125)",25.800989,28.722198,1.113221,25.633991,590.5,94.325901,121.586667,41,HT,0.833581,BFC_20_015HT,FCs
1,BFC_20_015HT_44,BFC_20_015HT_44_5,130,"(180.62342834472656, 131.61355590820312)",29.766766,30.941345,1.039459,116.370674,693.0,111.740114,116.698851,49,HT,0.697115,BFC_20_015HT,FCs
2,BFC_20_015HT_44,BFC_20_015HT_44_10,130,"(211.50559997558594, 111.84771728515625)",24.316208,30.800104,1.266649,166.917999,579.5,92.669046,122.059722,37,HT,0.847566,BFC_20_015HT,FCs
3,BFC_20_015HT_44,BFC_20_015HT_44_11,130,"(115.06068420410156, 90.58736419677734)",21.845362,25.631424,1.173312,78.253372,429.5,80.669047,125.513651,33,HT,0.828970,BFC_20_015HT,FCs
4,BFC_20_015HT_60,BFC_20_015HT_60_2,135,"(159.7198028564453, 313.9161071777344)",22.827908,37.206638,1.629875,63.613754,629.0,119.053823,119.956183,60,HT,0.557383,BFC_20_015HT,FCs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132739,PTC_18_114_C_7,PTC_18_114_C_7_4,150,"(25.23188018798828, 372.13726806640625)",21.507936,22.412224,1.042044,179.374008,371.5,91.497473,146.170274,47,PTC,0.557353,PTC_18_114_C,PTC
132740,PTC_18_114_C_7,PTC_18_114_C_7_10,150,"(344.8179626464844, 269.1244201660156)",16.191385,38.405937,2.371998,66.419403,275.0,119.740114,126.056469,59,PTC,0.240903,PTC_18_114_C,PTC
132741,PTC_18_114_C_7,PTC_18_114_C_7_79,150,"(204.18914794921875, 106.93700408935547)",16.766100,21.212500,1.265202,127.885872,271.0,67.254833,145.313492,33,PTC,0.752509,PTC_18_114_C,PTC
132742,PTC_18_114_C_7,PTC_18_114_C_7_87,150,"(381.6947326660156, 93.18396759033203)",24.777472,36.792564,1.484920,56.854118,663.0,118.710676,143.766590,60,PTC,0.590913,PTC_18_114_C,PTC


In [348]:
df_predict_3_64_merge = pd.merge(df_predict_3_64,merge,how="left",left_on="image_name",right_on="img_name")

In [334]:
df_predict_3_64

,image_name,predict_type
0,BFC_20_015HT_32_17,HT
1,BFC_20_015HT_46_20,Normal
2,BFC_20_015HT_46_26,CYST
3,BFC_20_015HT_7_26,HT
4,BFC_20_015HT_7_41,HT
...,...,...
15739,PTC_18_114_C_56_69,PTC
15740,PTC_18_114_C_56_85,PTC
15741,PTC_18_114_C_56_127,PTC
15742,PTC_18_114_C_32_73,PTC


In [350]:
df_predict_3_64_merge.describe()

,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,15744.000000,15744.000000,15744.000000,15744.000000,15744.000000,15744.000000,15744.000000,15744.000000,15744.000000,15744.000000
mean,138.781123,23.598795,30.128795,1.291296,89.446716,543.107342,96.213124,131.521504,44.444995,0.738286
std,7.464199,3.978266,5.640897,0.230884,52.014414,181.076353,17.918665,8.876591,10.292361,0.137416
min,115.000000,15.022781,15.942433,1.001183,0.000000,33.500000,51.698484,84.801952,20.000000,0.102792
25%,135.000000,20.841269,26.098567,1.129320,44.318330,417.500000,82.426406,125.281628,37.000000,0.675153
50%,140.000000,23.328151,29.188064,1.231330,88.883579,513.750000,93.497474,131.305272,43.000000,0.781690
75%,145.000000,26.169063,33.442426,1.387162,134.980438,648.500000,108.325901,137.652373,52.000000,0.838942
max,150.000000,51.460274,70.827164,3.828047,179.993591,1804.500000,190.166520,159.912917,75.000000,0.918743


In [351]:
df_predict_3_64_merge[df_predict_3_64_merge["real_type"].str.contains("HT")].describe()

,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,3551.000000,3551.000000,3551.000000,3551.000000,3551.000000,3551.000000,3551.000000,3551.000000,3551.000000,3551.000000
mean,131.427767,22.008403,27.013867,1.241752,90.672818,454.977894,86.885440,127.224480,39.896367,0.762507
std,2.917225,3.203851,3.579474,0.182654,52.070369,117.092250,13.319036,6.645120,8.562872,0.124462
min,125.000000,15.022781,16.703276,1.002923,0.017884,78.500000,55.455844,103.330952,20.000000,0.170969
25%,130.000000,19.778318,24.659687,1.113354,45.661316,379.500000,77.597979,122.681561,34.000000,0.713765
50%,130.000000,21.940350,26.573334,1.198069,90.594200,440.500000,84.669047,126.932602,39.000000,0.805126
75%,135.000000,23.987075,28.950784,1.320021,136.002571,515.000000,94.669047,131.733732,46.000000,0.849713
max,135.000000,33.851788,39.668457,2.412816,179.872650,997.500000,153.296463,152.835758,75.000000,0.916878


In [353]:
df_predict_3_64_merge[df_predict_3_64_merge["real_type"].str.contains("Normal")].describe()

,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,138.108016,21.845975,26.849830,1.242700,89.623608,449.179178,83.436386,129.521630,36.926291,0.805317
std,8.001267,2.872122,3.166480,0.178487,52.232367,98.629919,8.998164,9.206395,5.609958,0.083119
min,125.000000,15.052913,15.942433,1.002142,0.071162,119.500000,54.284271,94.417239,20.000000,0.207680
25%,125.000000,19.959597,24.802832,1.116115,44.139644,384.375000,77.497475,123.358626,33.000000,0.778631
50%,140.000000,21.857627,26.612831,1.198287,87.912899,444.000000,83.012192,129.108452,37.000000,0.829187
75%,145.000000,23.685279,28.687803,1.319673,135.473282,507.000000,89.012192,135.611582,41.000000,0.859855
max,145.000000,33.600616,39.880482,2.403821,179.874222,962.500000,119.740114,159.669384,50.000000,0.918743


In [354]:
df_predict_3_64_merge[df_predict_3_64_merge["real_type"].str.contains("HCN")].describe()

,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,4.0,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
mean,130.0,23.240380,26.192021,1.139948,63.441625,432.500000,85.754833,126.282790,37.250000,0.697292
std,10.0,4.887198,3.932451,0.100060,39.373170,229.835086,10.502099,13.032874,8.180261,0.256961
min,125.0,17.433847,22.200018,1.062824,16.577276,147.000000,76.426406,116.768308,27.000000,0.316097
25%,125.0,21.392076,23.662360,1.063204,38.527996,350.250000,80.047726,119.974891,33.000000,0.675412
50%,125.0,23.076982,25.674488,1.111791,67.859982,437.250000,82.961940,121.406471,38.000000,0.797492
75%,130.0,24.925286,28.204150,1.188535,92.773611,519.500000,88.669047,127.714370,42.250000,0.819371
max,145.0,29.373709,31.219091,1.273386,101.469261,708.500000,100.669047,145.549911,46.000000,0.878087


In [356]:
df_predict_3_64_merge[df_predict_3_64_merge["real_type"].str.contains("CYST")].describe()

,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,1542.000000,1542.000000,1542.000000,1542.000000,1542.000000,1542.000000,1542.000000,1542.000000,1542.000000,1542.000000
mean,142.876135,23.167587,27.934483,1.214762,88.421658,495.879053,88.458383,131.122262,40.068742,0.798025
std,11.142384,2.716391,3.451390,0.160432,51.003566,101.924476,11.235584,8.592791,7.515362,0.090162
min,115.000000,18.036036,18.875011,1.001183,0.134812,240.500000,63.941125,104.939287,21.000000,0.153842
25%,130.000000,21.080336,25.666598,1.095615,45.128002,422.000000,80.769552,125.483487,35.000000,0.767873
50%,150.000000,23.094261,27.558190,1.172895,86.601086,489.000000,86.911687,131.236265,39.000000,0.827565
75%,150.000000,25.043853,29.772593,1.294084,132.026371,556.375000,94.325901,136.677333,44.000000,0.857853
max,150.000000,34.494499,39.993793,2.036222,179.693146,962.000000,141.296463,159.912917,70.000000,0.907832


In [357]:
df_predict_3_64_merge[df_predict_3_64_merge["real_type"].str.contains("PTC")].describe()

,threshold,axle_short,axle_long,ratio,angle,area_cv2,perimeter,contours_pixel,contours_number,calcularity
count,7703.000000,7703.000000,7703.000000,7703.000000,7703.000000,7703.000000,7703.000000,7703.000000,7703.000000,7703.000000
mean,141.613008,25.088363,33.259232,1.348107,89.032591,629.144035,106.954001,134.349357,50.295210,0.689564
std,4.908715,4.287657,5.792899,0.264217,52.109358,200.188465,16.979244,8.682813,9.494307,0.148619
min,135.000000,15.035863,16.270050,1.002323,0.000000,33.500000,51.698484,84.801952,20.000000,0.102792
25%,140.000000,22.289876,29.615429,1.161279,43.652267,505.000000,96.083260,128.313685,44.000000,0.613885
50%,140.000000,25.164970,32.893864,1.284271,88.702560,628.000000,106.911687,134.693307,51.000000,0.725243
75%,140.000000,27.913318,36.495378,1.459787,134.842354,746.750000,117.639609,140.381936,58.000000,0.802288
max,150.000000,51.460274,70.827164,3.828047,179.993591,1804.500000,190.166520,154.994865,67.000000,0.911287


In [355]:
df_predict_3_64_merge['real_type'].value_counts()

real_type
PTC       7703
HT        3551
Normal    2944
CYST      1542
HCN          4
Name: count, dtype: int64

In [106]:
df_predict_3_64.to_excel("./df_predict_3_64.xlsx",index=False)

In [284]:
df_predict_3_64

,image_name,predict_type
0,BFC_20_002_98_2,Normal
1,BFC_20_002_98_3,Normal
2,BFC_20_002_98_5,Normal
3,BFC_20_002_98_6,Normal
4,BFC_20_002_98_11,Normal
5,BFC_20_002_98_16,CYST
6,BFC_20_002_98_19,HT
7,BFC_20_003_10_7,Normal
8,BFC_20_003_10_8,Normal
9,BFC_20_003_10_12,Normal


In [338]:
df_predict_3_64.loc[df_predict_3_64["image_name"].str.contains("CYST"),"real_type"] = "CYST"
df_predict_3_64.loc[df_predict_3_64["image_name"].str.contains("HT"),"real_type"] = "HT"
df_predict_3_64.loc[df_predict_3_64["image_name"].str.contains("PTC"),"real_type"] = "PTC"
df_predict_3_64.loc[df_predict_3_64["image_name"].str.contains("HCN"),"real_type"] = "HCN"
df_predict_3_64.loc[df_predict_3_64["real_type"].isna(),"real_type"] = "Normal"

In [339]:
df_predict_3_64.loc[df_predict_3_64["predict_type"]==df_predict_3_64["real_type"],"correct"] = "Y"
df_predict_3_64.loc[df_predict_3_64["correct"].isna(),"correct"] = "N"

In [340]:
df_predict_3_64['mass']=df_predict_3_64['image_name'].apply(lambda x : "_".join(x.split("_")[0:-1]))
df_predict_3_64['slide']=df_predict_3_64['mass'].apply(lambda x : "_".join(x.split("_")[0:-1]))
group = df_predict_3_64.groupby(['mass','predict_type']).size().reset_index(name='count')
category_max = group.groupby('mass').apply(lambda x: x.loc[x['count'].idxmax()])
category_max = category_max[['mass', 'predict_type', 'count']].reset_index(drop=True)

In [341]:
category_max.loc[category_max["mass"].str.contains("CYST"),"real_type"] = "CYST"
category_max.loc[category_max["mass"].str.contains("HT"),"real_type"] = "HT"
category_max.loc[category_max["mass"].str.contains("PTC"),"real_type"] = "PTC"
category_max.loc[category_max["mass"].str.contains("HCN"),"real_type"] = "HCN"
category_max.loc[category_max["real_type"].isna(),"real_type"] = "Normal"

In [342]:
category_max.loc[category_max["predict_type"]==category_max["real_type"],"correct"] = "Y"
category_max.loc[category_max["correct"].isna(),"correct"] = "N"
category_max['slide']=category_max['mass'].apply(lambda x : "_".join(x.split("_")[0:-1]))

In [343]:
final_result = category_max.groupby(['slide','predict_type']).size().reset_index(name='count')
final_result_max = final_result.groupby('slide').apply(lambda x: x.loc[x['count'].idxmax()])

In [344]:
final_result_max.loc[final_result_max["slide"].str.contains("CYST"),"real_type"] = "CYST"
final_result_max.loc[final_result_max["slide"].str.contains("HT"),"real_type"] = "HT"
final_result_max.loc[final_result_max["slide"].str.contains("PTC"),"real_type"] = "PTC"
final_result_max.loc[final_result_max["slide"].str.contains("HCN"),"real_type"] = "HCN"
final_result_max.loc[final_result_max["real_type"].isna(),"real_type"] = "Normal"

In [345]:
final_result_max.loc[final_result_max["predict_type"]==final_result_max["real_type"],"correct"] = "Y"
final_result_max.loc[final_result_max["correct"].isna(),"correct"] = "N"

In [346]:
final_result_max['correct'].value_counts()

correct
Y    154
N      2
Name: count, dtype: int64

In [347]:
154/156

0.9871794871794872

# 3 layer 32

In [103]:
train_images = np.array(train_images)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_images = np.array(test_images)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Create and train the cell image classifier
classifier = CellImageClassifier(image_width, image_height, num_channels, num_classes)
classifier.train_3_32(train_images, train_labels, num_epochs, batch_size)

# Evaluate the classifier
classifier.evaluate_3_32(test_images, test_labels)

Epoch 1/10
3250/3250 [==============================] - 532s 163ms/step - loss: 0.6258 - accuracy: 0.7044 - auc_20: 0.9257 - precision_20: 0.8565 - recall_20: 0.5476 - true_positives_20: 104000.0000 - true_negatives_20: 0.0000e+00 - false_positives_20: 312000.0000 - false_negatives_20: 0.0000e+00 - precision_at_recall_20: 0.9134 - sensitivity_at_specificity_20: 0.9982 - specificity_at_sensitivity_20: 0.9841
Epoch 2/10
3250/3250 [==============================] - 515s 159ms/step - loss: 0.5262 - accuracy: 0.7616 - auc_20: 0.9481 - precision_20: 0.8451 - recall_20: 0.6639 - true_positives_20: 104000.0000 - true_negatives_20: 3.0000 - false_positives_20: 311997.0000 - false_negatives_20: 0.0000e+00 - precision_at_recall_20: 0.9673 - sensitivity_at_specificity_20: 0.9994 - specificity_at_sensitivity_20: 0.9944
Epoch 3/10
3250/3250 [==============================] - 537s 165ms/step - loss: 0.4920 - accuracy: 0.7789 - auc_20: 0.9550 - precision_20: 0.8436 - recall_20: 0.7016 - true_positives

In [104]:
# Predict on new images
df_predict_3_32 = pd.DataFrame()
for i in predict_data['img_name']:
    new_image_path = f'./all_image/all_singal/{i}.png'
    predicted_class_index = classifier.predict_3_64(new_image_path)

    # Map class index to class label (assuming binary classification)
    class_labels = ['CYST', 'HT', 'PTC', 'Normal']
    predicted_class_label = class_labels[predicted_class_index]
    predict = {"image_name":i,"predict_type":predicted_class_label}
    
    df_predict_3_32 = pd.concat([df_predict_3_32,pd.DataFrame([predict])],ignore_index=True)

In [107]:
df_predict_3_32.to_excel("./df_predict_3_32.xlsx",index=False)

In [108]:
df_predict_3_32

,image_name,predict_type
0,BFC_20_015HT_44_1,Normal
1,BFC_20_015HT_44_11,Normal
2,BFC_20_015HT_60_4,Normal
3,BFC_20_015HT_87_13,Normal
4,BFC_20_015HT_87_20,Normal
...,...,...
15739,PTC_18_114_C_56_137,Normal
15740,PTC_18_114_C_32_68,Normal
15741,PTC_18_114_C_7_4,Normal
15742,PTC_18_114_C_7_10,Normal


# 3 layer32 size=32

In [136]:
train_images = np.array(train_images)
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
test_images = np.array(test_images)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Create and train the cell image classifier
classifier = CellImageClassifier(image_width, image_height, num_channels, num_classes)
classifier.train_3_32(train_images, train_labels, num_epochs, batch_size)

# Evaluate the classifier
classifier.evaluate_3_32(test_images, test_labels)

Epoch 1/10
3250/3250 [==============================] - 327s 100ms/step - loss: 0.6194 - accuracy: 0.7119 - auc_27: 0.9277 - precision_27: 0.8599 - recall_27: 0.5612 - true_positives_27: 104000.0000 - true_negatives_27: 0.0000e+00 - false_positives_27: 312000.0000 - false_negatives_27: 0.0000e+00 - precision_at_recall_27: 0.9228 - sensitivity_at_specificity_27: 0.9986 - specificity_at_sensitivity_27: 0.9860
Epoch 2/10
3250/3250 [==============================] - 370s 114ms/step - loss: 0.5372 - accuracy: 0.7530 - auc_27: 0.9456 - precision_27: 0.8416 - recall_27: 0.6540 - true_positives_27: 104000.0000 - true_negatives_27: 0.0000e+00 - false_positives_27: 312000.0000 - false_negatives_27: 0.0000e+00 - precision_at_recall_27: 0.9647 - sensitivity_at_specificity_27: 0.9995 - specificity_at_sensitivity_27: 0.9939
Epoch 3/10
3250/3250 [==============================] - 268s 82ms/step - loss: 0.5078 - accuracy: 0.7702 - auc_27: 0.9516 - precision_27: 0.8390 - recall_27: 0.6896 - true_positi

In [137]:
# Predict on new images
df_predict_3_32 = pd.DataFrame()
for i in predict_data['img_name']:
    new_image_path = f'./all_image/all_singal/{i}.png'
    predicted_class_index = classifier.predict_3_64(new_image_path)

    # Map class index to class label (assuming binary classification)
    class_labels = ['CYST', 'HT', 'PTC', 'Normal']
    predicted_class_label = class_labels[predicted_class_index]
    predict = {"image_name":i,"predict_type":predicted_class_label}
    
    df_predict_3_32 = pd.concat([df_predict_3_32,pd.DataFrame([predict])],ignore_index=True)

In [138]:
df_predict_3_32

,image_name,predict_type
0,BFC_20_015HT_44_1,PTC
1,BFC_20_015HT_44_11,PTC
2,BFC_20_015HT_60_4,Normal
3,BFC_20_015HT_87_13,PTC
4,BFC_20_015HT_87_20,PTC
...,...,...
15739,PTC_18_114_C_56_137,Normal
15740,PTC_18_114_C_32_68,Normal
15741,PTC_18_114_C_7_4,PTC
15742,PTC_18_114_C_7_10,Normal
